In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pyspark.sql.functions as f

In [2]:
import os
ip_address = os.environ.get('IP', '10.0.0.4')
print(ip_address)

10.0.0.4


In [3]:
# Configurações de acesso ao postgres
host = ip_address
port = "5432"
database = "postgres"
url = f"jdbc:postgresql://{host}:{port}/{database}"

In [4]:
spark = SparkSession.builder \
    .appName("Credit Events Processor") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.23") \
    .getOrCreate()

In [11]:
import datetime
now = datetime.datetime.now()
# Calcule o lowerBound como 24 horas antes de now
lowerBound_ = now - datetime.timedelta(days=1)
upperBound_ = now
print("lowerBound:", lowerBound_)
print("upperBound:", upperBound_)

lowerBound: 2023-09-28 20:31:37.232947
upperBound: 2023-09-29 20:31:37.232947


In [ ]:
"""Exercicio 1:
    1. Conseguir buscar o `lowerBound` e `upperBound` com uma consulta de max e mim. select max(trans_date), min(trans_date) from public.credit_events ce;
    2. Salvar os valores em lowerBound_ e upperBound_
"""
print("lowerBound:", lowerBound_)
print("upperBound:", upperBound_)

In [21]:
df = ( 
    spark.read
         .format("jdbc")
         .options(url=url, driver="org.postgresql.Driver", dbtable="credit_events", user="postgres", password="postgres") \
         .options(partitionColumn="trans_date", lowerBound=lowerBound_, upperBound=upperBound_, numPartitions="30")
         .load()
)

In [30]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- cpf: long (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- trans_num: long (nullable = true)
 |-- trans_date: timestamp (nullable = true)
 |-- trans_time: timestamp (nullable = true)
 |-- unix_time: long (nullable = true)
 |-- category: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- value: decimal(10,2) (nullable = true)
 |-- location: string (nullable = true)
 |-- partition_date: string (nullable = true)



In [23]:
df.count()

200000

In [ ]:
(
    df
    .write
    .mode("overwrite")
    .parquet("/home/jovyan/work/datalake/credit_events")
)

In [ ]:
"""Exercicio 2:
    1. Escrever no datalake/credit_events os eventos particionados por "partition_date". Utilizar `.partitionBy("partition_date").`
    2. Atualizar os dados que serão escritos em D-1, ou seja, sempre inserir os dados das últimas 24h. Deve ser feito um filter("partition_date BETWEEN 'lower_date' AND 'upper_date'")
"""

from pyspark.sql.functions import date_format

df = df.withColumn("partition_date", date_format("trans_date", "yyyy-MM-dd"))
